# What do the functions $H^{(1)}_\nu(Zr)$ and $H^{(2)}_\nu(Zr)$ (and linear combinations) look like on regions we're looking at.

These linear combos form solutions of radial part of step index fiber modes in unbounded domain.  We only ever use H1 so we get outgoing sols, but what do they look like in general?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
from ipywidgets import interactive, FloatSlider, Layout
from fiberamp.fiber.microstruct.bragg import BraggExact


In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

In [ ]:
ts = [4.0775e-05, 2e-5, 1e-5]
rhos =[sum(ts[:i]) for i in range(1, len(ts)+1)]
mats = ['air', 'glass', 'glass']
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_glass]
maxhs = [.1,.03, .02]
pml = {'alpha':.4, 'R0':rhos[-2]}

In [ ]:
A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, wl=1.4e-6)


In [ ]:
k_low = A.ks[0] * A.L

In [ ]:
beta = 67.33269141518103+9.422996804515637e-05j

In [ ]:
2 * beta.real * beta.imag, k_low**2-beta.real**2 + beta.imag**2

In [ ]:
K = np.sqrt(k_low**2 - beta**2, dtype=complex)
K

In [ ]:
-K.imag/K.real

In [ ]:
def hankel1_asymp(nu, z):
    return np.sqrt(2/(np.pi * z), dtype=complex) * np.e**(1j*(z - nu/2 * np.pi - np.pi/4))

In [ ]:
def hankel2_asymp(nu, z):
    return np.sqrt(2/(np.pi * z), dtype=complex) * np.e**(-1j*(z - nu/2 * np.pi - np.pi/4))

In [ ]:
nu = 1
R = 1
R_out = 40

k1 = A.ks[0] * A.scale
k2 = A.ks[1] * A.scale

def hankel1_plot(beta_re, beta_im):
    plt.figure(2, figsize=(14,8))
    
    k2 = A.ks[1] * A.scale
    beta = beta_re + beta_im * 1j
    K = np.sqrt(k2**2 - beta**2, dtype=complex)
    
    rs = np.arange(R, R_out, .01)
#     h1s = sp.hankel1(1, K * rs)
    h1s = hankel1_asymp(1, K * rs)
    print(K)
    
    plt.ylim(-10,10)
    plt.title("$H^{(1)}_\\nu(\,Zr)$ for $\\nu=1$ \n and $Z^2 = k_{clad}^2 - \\beta^2$ \n")
    plt.plot(rs, h1s.real, label='real')
    plt.plot(rs, h1s.imag, '--', label='imag')

    plt.legend()
    plt.show()

m, M = 1.0001* k2, k1
interactive_plot = interactive(hankel1_plot,
                               beta_re=FloatSlider(min=m, max=M, step=(M-m)/201, value=(m),
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               beta_im=FloatSlider(min=-.1, max=.1, step=0.00, value=0,
                                                   readout_format='.3f', layout=Layout(width='90%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

In [ ]:
nu = 1
R = 1
R_out = 20

k1 = A.ks[0] * A.scale
k2 = A.ks[1] * A.scale

def hankel2_plot(beta_re, beta_im):
    plt.figure(2, figsize=(14,8))
    
    k2 = A.ks[1] * A.scale
    beta = beta_re + beta_im * 1j
    K = np.sqrt(k2**2 - beta**2, dtype=complex)
    
    rs = np.arange(R, R_out, .001)
    h2s = sp.hankel2(1, K * rs)
#     h2s = hankel2_asymp(1, K * rs)
    
    plt.ylim(-10,10)
    plt.title("$H^{(2)}_\\nu(\,Zr)$ for $\\nu=1$ \n and $Z^2 = k_{clad}^2 - \\beta^2$ \n")
    plt.plot(rs, h2s.real, label='real')
    plt.plot(rs, h2s.imag, '--', label='imag')
    plt.legend()
    plt.show()

m, M = 1.000001*k2, k1
interactive_plot = interactive(hankel2_plot,
                               beta_re=FloatSlider(min=m, max=M, step=(M-m)/501, value=(1.00001*k2),
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               beta_im=FloatSlider(min=-.1, max=.1, step=0.00, value=0,
                                                   readout_format='.3f', layout=Layout(width='90%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

# Including PML

Now we include PML after a certain point for H^2

In [ ]:
nu = 1

Rm = 1
RM = 4

k1 = A.ks[0] * A.scale

def hankel2_plot(beta_re, beta_im, alpha, R):
    plt.figure(2, figsize=(14,8))
    
    k2 = A.ks[1] * A.scale
    beta = beta_re + beta_im * 1j
    gamma = np.sqrt(beta**2 - k2**2, dtype=complex)
    
    rs = np.arange(Rm, RM, .01, dtype=complex)
    rs[np.where(rs.real>=R)] += (rs[np.where(rs.real>=R)] - R) * (-alpha) * 1j
    h2s = -sp.hankel2(1, 1j * gamma * rs)
    
    plt.ylim(-1,1)
    plt.title("$H^{(2)}_\\nu(\,Zr)$ for $\\nu=1$ \n and $Z^2 = k_{clad}^2 - \\beta^2$ \n")
    plt.plot(rs.real, h2s.real, label='real')
    plt.plot(rs.real, h2s.imag, '--', label='imag')
    minL, maxL = plt.gca().get_ylim()
    plt.plot([R,R], [minL, maxL], linestyle=':', color='g', linewidth=.9)
    plt.legend()
    plt.show()

m, M = .99*k1, 1.01*k1
interactive_plot = interactive(hankel2_plot,
                               beta_re=FloatSlider(min=m, max=M, step=(M-m)/51, value=(k1),
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               beta_im=FloatSlider(min=-.3, max=.3, step=0.01, value=-1e-5,
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               alpha=FloatSlider(min=0, max=2, step=0.002, value=0.06,
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               R=FloatSlider(min=1.01*Rm, max=.99*RM, step=(RM-Rm)/41, value=(RM+Rm)/2,
                                                   readout_format='.3f', layout=Layout(width='90%'))                              )
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

# Asymptotic Forms

In [ ]:
def hankel2_asymp(nu, z):
    return np.sqrt(2/(np.pi * z), dtype=complex) * np.e**(-1j*(z - nu/2 * np.pi - np.pi/4))

In [ ]:
nu = 1

Rm = .1
RM = 1

k1 = A.ks[0] * A.scale

def hankel2_asymp_plot(beta_re, beta_im, alpha, R):
    plt.figure(2, figsize=(14,8))
    
    k2 = A.ks[1] * A.scale
    beta = beta_re + beta_im * 1j
    K = np.sqrt(k2**2 - beta**2, dtype=complex)
    
    rs = np.arange(Rm, RM, .001, dtype=complex)
    rs[np.where(rs.real>=R)] += (rs[np.where(rs.real>=R)] - R) * (-alpha) * 1j
    
    h2s = sp.hankel2(1, K * rs)
    h2s2 = hankel2_asymp(1, K * rs)
    plt.ylim(-.6,.6)
    plt.title("$H^{(2)}_\\nu(\,Zr)$ for $\\nu=1$ \n and $Z^2 = k_{clad}^2 - \\beta^2$ \n")
    
    plt.plot(rs.real, h2s.real, label='real_hankel')
    plt.plot(rs.real, h2s2.real, linestyle=(0,(4,8)), label='real_asymp')

    plt.plot(rs.real, h2s.imag, label='imag_hankel')
    plt.plot(rs.real, h2s2.imag,  linestyle=(0,(4,8)), label='imag_asymp')

    minL, maxL = plt.gca().get_ylim()
    plt.plot([R,R], [minL, maxL], linestyle=':', color='g', linewidth=.9)
    plt.legend()
    plt.show()

m, M = k1, k2
interactive_plot = interactive(hankel2_asymp_plot,
                               beta_re=FloatSlider(min=m, max=M, step=(M-m)/51, value=(k1),
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               beta_im=FloatSlider(min=-.3, max=.3, step=0.01, value=0.01,
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               alpha=FloatSlider(min=0, max=.2, step=0.002, value=0.004,
                                                   readout_format='.3f', layout=Layout(width='90%')),
                               R=FloatSlider(min=1.01*Rm, max=.99*RM, step=(RM-Rm)/41, value=(RM+Rm)/2,
                                                   readout_format='.3f', layout=Layout(width='90%'))                              )
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot